In [1]:
# Necessary imports

from sklearn.linear_model import LinearRegression
import pandas as pd
import cv2
import os
import io
import numpy as np
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import torch
from torchvision.models.segmentation import deeplabv3_resnet101
from torchvision import transforms
import joblib

In [2]:
# Measurements file of subjects

df_measurements = pd.read_csv('train/measurements.csv')
df_measurements.head(5)

subject_id      ankle  arm-length  \
0  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg  24.343954   52.902378   
1  -5OHO2b5mRDY1rvAf07sYYtekEEX7ViEGmvRr7l4tfc  22.001783   46.763233   
2  -5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI  26.565006   53.288597   
3  -60o95b9oEE83BDV3GpsAVPA4TFvxuPPqtXEY5PApy8  24.779566   49.077751   
4  -67TrwlJ3GSsqPeHaa2Nb0iQ-BKZfIx4ERysQPZ0-SY  20.788853   42.229557   

       bicep       calf       chest    forearm      height         hip  \
0  33.294395  38.709511  102.526024  27.599094  177.610992  108.047134   
1  28.288971  35.649963   99.253395  24.813375  164.379623   92.128685   
2  30.800959  39.854950  102.391273  27.719795  183.718842  106.044121   
3  34.613113  38.104267  111.164070  28.550409  169.059113  109.529465   
4  24.527142  29.181356   93.627777  21.979038  146.516632   87.134102   

   leg-length  shoulder-breadth  shoulder-to-crotch      thigh       waist  \
0   81.736435         35.915691           67.891273  57.872795   92.286171   
1   75.127258         33.848778           60.541386  52.418320   87.307899   
2   84.055527         37.856457           72.004379  52.934086   96.596413   
3   74.310234         38.197800           68.536713  58.656673  103.395119   
4   67.269859         31.659197           52.697277  41.844944   74.400856   

       wrist  
0  16.709131  
1  15.192038  
2  17.936598  
3  17.029860  
4  15.120061

In [3]:
# Metadata for subjects

df_info = pd.read_csv('train/hwg_metadata.csv')
df_info.head(5)

subject_id  gender  height_cm  weight_kg
0  AhfyGaD1BKo0-u6PeQk2oI-Bw5l0XxpAqNroFuZuy0I  female     160.00       92.4
1  E8tDQ5IgdBwbjLIXu7GwcupEHYbi0eB18YNcsgsSbP8  female     175.75      102.8
2                     59bc2767681516389ccf45f9    male     174.80      106.9
3  nRatvgqbYTN77typTgGm4bAurxzpvf5nYg2Mb3gdh8c    male     181.50      111.8
4  fkq68ww6bdyKT16FavI1x_G-u7YZ9SoT1GNMUTSJUek  female     161.60       93.0

In [4]:
# Subject to photo mapping

df_photo_map = pd.read_csv('train/subject_to_photo_map.csv')
df_photo_map.head(5)

subject_id  \
0  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
1  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
2  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
3  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
4  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   

                           photo_id  
0  e6f404ebda41ebe93573d3e219c88297  
1  39a4241cb892618a694747e224d57b9e  
2  a00a6c0c074efbd92c894cbc77023939  
3  7c99b3017f21f9797851e70de084bfcb  
4  3000cb26b2090db5a7d82926af8c6a83

In [5]:
# Merge Measurements with Metadata

df_subject_info_merged = pd.merge(df_measurements, df_info, on='subject_id', how='inner')
df_subject_info_merged.head(5)

subject_id      ankle  arm-length  \
0  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg  24.343954   52.902378   
1  -5OHO2b5mRDY1rvAf07sYYtekEEX7ViEGmvRr7l4tfc  22.001783   46.763233   
2  -5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI  26.565006   53.288597   
3  -60o95b9oEE83BDV3GpsAVPA4TFvxuPPqtXEY5PApy8  24.779566   49.077751   
4  -67TrwlJ3GSsqPeHaa2Nb0iQ-BKZfIx4ERysQPZ0-SY  20.788853   42.229557   

       bicep       calf       chest    forearm      height         hip  \
0  33.294395  38.709511  102.526024  27.599094  177.610992  108.047134   
1  28.288971  35.649963   99.253395  24.813375  164.379623   92.128685   
2  30.800959  39.854950  102.391273  27.719795  183.718842  106.044121   
3  34.613113  38.104267  111.164070  28.550409  169.059113  109.529465   
4  24.527142  29.181356   93.627777  21.979038  146.516632   87.134102   

   leg-length  shoulder-breadth  shoulder-to-crotch      thigh       waist  \
0   81.736435         35.915691           67.891273  57.872795   92.286171   
1   75.127258         33.848778           60.541386  52.418320   87.307899   
2   84.055527         37.856457           72.004379  52.934086   96.596413   
3   74.310234         38.197800           68.536713  58.656673  103.395119   
4   67.269859         31.659197           52.697277  41.844944   74.400856   

       wrist  gender  height_cm  weight_kg  
0  16.709131    male      178.0       79.6  
1  15.192038  female      163.0       65.0  
2  17.936598    male      185.0       82.9  
3  17.029860    male      166.0       88.7  
4  15.120061  female      144.6       44.2

In [6]:
# Method to return the image path of subjects of front view and left view

def return_image_path(photo_id):
    image_dir = 'train/mask'
    image_path_mask = os.path.join(image_dir, f'{photo_id}.png')
    
    
    image_dir = 'train/mask_left'
    image_path_maskleft = os.path.join(image_dir, f'{photo_id}.png')
    
    image = cv2.imread(image_path_maskleft)
    if image is None:
        print(f'Warning: Image {photo_id}.png not found')
    return [image_path_mask, image_path_maskleft]

In [7]:
#Extract features from imagges

def extract_silhouette_features(photo_id):
    if '.png' in photo_id:
        image_ = [photo_id]
    else:
        image_ = return_image_path(photo_id=photo_id)
    features = {}
    loop = 0
    for image in image_:
        # Convert image to grayscale
        image = cv2.imread(image)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        if loop == 0:
            loop = 1
        # Thresholding to get binary silhouette
        ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
           
        # Iterate through contours
        for contour in contours:
            # Calculate area and perimeter
            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, True)
            
            # Calculate bounding box
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h if h != 0 else 0.0
            
            # Calculate solidity (area / convex hull area)
            hull = cv2.convexHull(contour)
            hull_area = cv2.contourArea(hull)
            solidity = float(area) / hull_area if hull_area != 0 else 0.0
            
            # Add features to dictionary
            if loop == 2:
                features['area_left'] = area
                features['perimeter_left'] = perimeter
                features['aspect_ratio_left'] = aspect_ratio
                features['solidity_left'] = solidity
            else:
                features['area'] = area
                features['perimeter'] = perimeter
                features['aspect_ratio'] = aspect_ratio
                features['solidity'] = solidity
            
        loop = 2
    
    return features

In [8]:
# Extract image features and append 

df_photo_map['metrics'] = df_photo_map['photo_id'].apply(extract_silhouette_features)
df_photo_map.head(10)

subject_id  \
0  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
1  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
2  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
3  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
4  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
5  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
6  -5OHO2b5mRDY1rvAf07sYYtekEEX7ViEGmvRr7l4tfc   
7  -5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI   
8  -5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI   
9  -60o95b9oEE83BDV3GpsAVPA4TFvxuPPqtXEY5PApy8   

                           photo_id  \
0  e6f404ebda41ebe93573d3e219c88297   
1  39a4241cb892618a694747e224d57b9e   
2  a00a6c0c074efbd92c894cbc77023939   
3  7c99b3017f21f9797851e70de084bfcb   
4  3000cb26b2090db5a7d82926af8c6a83   
5  e62a945f9950f739ea73b90438a9e9cc   
6  69748a13010a169df0f7cb986ac1129e   
7  0d4a9cce6294a2bdc6dae93d0d9c42a8   
8  cb5e3a2ce44d28764b2b453b0950e980   
9  ecff10b191d87ec602eb58621d5a8e23   

                                             metrics  
0  {'area': 137783.5, 'perimeter': 4272.553720951...  
1  {'area': 123261.0, 'perimeter': 4039.801413536...  
2  {'area': 125410.0, 'perimeter': 4145.874570727...  
3  {'area': 118977.5, 'perimeter': 4024.661367297...  
4  {'area': 118898.5, 'perimeter': 3985.666420459...  
5  {'area': 128925.0, 'perimeter': 4148.477602839...  
6  {'area': 124265.5, 'perimeter': 3948.293837547...  
7  {'area': 112290.5, 'perimeter': 3929.416634917...  
8  {'area': 111469.0, 'perimeter': 3927.090739607...  
9  {'area': 134284.5, 'perimeter': 3989.097875595...

In [9]:
# Extract each feature from the dict 

df_metrics = pd.json_normalize(df_photo_map['metrics'])

In [10]:
# Map extracted features for each image wrt to subject

df_photo_map_extended = pd.concat([df_photo_map[['subject_id', 'photo_id']], df_metrics], axis=1)
df_photo_map_extended.head(5)

subject_id  \
0  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
1  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
2  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
3  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   
4  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg   

                           photo_id      area    perimeter  aspect_ratio  \
0  e6f404ebda41ebe93573d3e219c88297  137783.5  4272.553721      0.530201   
1  39a4241cb892618a694747e224d57b9e  123261.0  4039.801414      0.535161   
2  a00a6c0c074efbd92c894cbc77023939  125410.0  4145.874571      0.637850   
3  7c99b3017f21f9797851e70de084bfcb  118977.5  4024.661367      0.644737   
4  3000cb26b2090db5a7d82926af8c6a83  118898.5  3985.666420      0.606538   

   solidity  area_left  perimeter_left  aspect_ratio_left  solidity_left  
0  0.487090    74067.0     2104.396091           0.177246       0.712320  
1  0.489721    72330.0     2136.195081           0.182588       0.669973  
2  0.422959    45887.5     1707.501649           0.182236       0.661618  
3  0.416841    74217.5     2200.579858           0.185273       0.639856  
4  0.443443    76588.5     2214.035701           0.173759       0.676206

In [11]:
# Drop column photid

df_photo_map_extended = df_photo_map_extended.drop(columns=['photo_id'])

In [12]:
# Avg the metrics for each subject since some subjects have multiple images in dataset

df_final_one = df_photo_map_extended.groupby('subject_id').mean().reset_index()
df_final_one.head(5)

subject_id           area    perimeter  \
0  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg  125542.583333  4102.839183   
1  -5OHO2b5mRDY1rvAf07sYYtekEEX7ViEGmvRr7l4tfc  124265.500000  3948.293838   
2  -5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI  111879.750000  3928.253687   
3  -60o95b9oEE83BDV3GpsAVPA4TFvxuPPqtXEY5PApy8  133137.250000  3980.520712   
4  -67TrwlJ3GSsqPeHaa2Nb0iQ-BKZfIx4ERysQPZ0-SY  127530.000000  3978.428829   

   aspect_ratio  solidity  area_left  perimeter_left  aspect_ratio_left  \
0      0.591337  0.451070   69946.50     2096.124013           0.179143   
1      0.654762  0.428414   91774.50     2435.172783           0.215909   
2      0.614127  0.432407   68101.25     2260.009834           0.206949   
3      0.592995  0.483474   86454.75     2309.992588           0.188049   
4      0.619385  0.468516   89461.50     2544.226604           0.219114   

   solidity_left  
0       0.672697  
1       0.679564  
2       0.623788  
3       0.728562  
4       0.664346

In [13]:
# Merging info df with finalone to extract info of subjects 
df_for_merge = df_subject_info_merged[['subject_id', 'weight_kg', 'height', 'wrist', 'waist', 'gender', 'hip']]

df = pd.merge(df_final_one, df_for_merge, on='subject_id', how='inner')

In [14]:
# Convert categorical col gender to numerical

df['gender'] = df['gender'].apply(lambda x:1 if x =='male' else 0)

In [15]:
df.head(5)

subject_id           area    perimeter  \
0  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg  125542.583333  4102.839183   
1  -5OHO2b5mRDY1rvAf07sYYtekEEX7ViEGmvRr7l4tfc  124265.500000  3948.293838   
2  -5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI  111879.750000  3928.253687   
3  -60o95b9oEE83BDV3GpsAVPA4TFvxuPPqtXEY5PApy8  133137.250000  3980.520712   
4  -67TrwlJ3GSsqPeHaa2Nb0iQ-BKZfIx4ERysQPZ0-SY  127530.000000  3978.428829   

   aspect_ratio  solidity  area_left  perimeter_left  aspect_ratio_left  \
0      0.591337  0.451070   69946.50     2096.124013           0.179143   
1      0.654762  0.428414   91774.50     2435.172783           0.215909   
2      0.614127  0.432407   68101.25     2260.009834           0.206949   
3      0.592995  0.483474   86454.75     2309.992588           0.188049   
4      0.619385  0.468516   89461.50     2544.226604           0.219114   

   solidity_left  weight_kg      height      wrist       waist  gender  \
0       0.672697       79.6  177.610992  16.709131   92.286171       1   
1       0.679564       65.0  164.379623  15.192038   87.307899       0   
2       0.623788       82.9  183.718842  17.936598   96.596413       1   
3       0.728562       88.7  169.059113  17.029860  103.395119       1   
4       0.664346       44.2  146.516632  15.120061   74.400856       0   

          hip  
0  108.047134  
1   92.128685  
2  106.044121  
3  109.529465  
4   87.134102

In [16]:
# Normalize data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Select the columns to normalize
columns_to_normalize = ['area','height', 'weight_kg','perimeter', 'aspect_ratio', 'solidity', 'area_left',
                         'perimeter_left', 'aspect_ratio_left', 'solidity_left']

# Fit and transform the data
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

In [17]:
df.head(5)

subject_id      area  perimeter  \
0  -494U-YoXOD8e8gkCuyaRLn4MLo5P8Dm2B1s59WBGdg  0.371689   1.093387   
1  -5OHO2b5mRDY1rvAf07sYYtekEEX7ViEGmvRr7l4tfc  0.281391   0.320375   
2  -5iHTazqnmWBsnUY6ymoCw1kORMymRDaUla1eNOjgvI -0.594352   0.220137   
3  -60o95b9oEE83BDV3GpsAVPA4TFvxuPPqtXEY5PApy8  0.908675   0.481569   
4  -67TrwlJ3GSsqPeHaa2Nb0iQ-BKZfIx4ERysQPZ0-SY  0.512210   0.471105   

   aspect_ratio  solidity  area_left  perimeter_left  aspect_ratio_left  \
0      0.593315 -1.061744  -0.695302       -0.631796          -0.249887   
1      1.236983 -1.487022   0.526485        0.405200           0.016917   
2      0.824603 -1.412060  -0.798587       -0.130544          -0.048106   
3      0.610142 -0.453481   0.228721        0.022331          -0.185261   
4      0.877965 -0.734267   0.397019        0.738747           0.040176   

   solidity_left  weight_kg    height      wrist       waist  gender  \
0      -0.314721   0.276159  0.585527  16.709131   92.286171       1   
1      -0.171355  -0.609718 -0.833141  15.192038   87.307899       0   
2      -1.335699   0.476392  1.240410  17.936598   96.596413       1   
3       0.851470   0.828315 -0.331406  17.029860  103.395119       1   
4      -0.489040  -1.871790 -2.748410  15.120061   74.400856       0   

          hip  
0  108.047134  
1   92.128685  
2  106.044121  
3  109.529465  
4   87.134102

In [18]:
# Model for wrist size calculation

# model_wrist = LinearRegression()
# model_wrist.fit(df.drop(columns=['subject_id', 'waist', 'wrist','hip']), df[['wrist']])
# model_wrist_file = "models/wrist_linear_regression_model.pkl"
# joblib.dump(model_wrist, model_wrist_file)

In [19]:
model_wrist_file = "models/wrist_linear_regression_model.pkl"
model_wrist = joblib.load(model_wrist_file)

In [20]:
# Model for waist size calculation

# model_waist = LinearRegression()
# model_waist.fit(df.drop(columns=['subject_id', 'waist', 'wrist','hip']), df[['waist']])
# model_waist_file = "models/waist_linear_regression_model.pkl"
# joblib.dump(model_waist, model_waist_file)


In [21]:
model_waist_file = "models/waist_linear_regression_model.pkl"
model_waist = joblib.load(model_waist_file)

In [22]:
# Model for hip size calculation

# model_hip = LinearRegression()
# model_hip.fit(df.drop(columns=['subject_id', 'waist', 'wrist','hip']), df[['hip']])
# model_hip_file = "models/hip_linear_regression_model.pkl"
# joblib.dump(model_hip, model_hip_file)

In [23]:
model_hip_file = "models/hip_linear_regression_model.pkl"
model_hip = joblib.load(model_hip_file)

In [24]:
# Method to extract features for inference images

def extract_silhouette_features_for_inference(image):
    image_array = np.array(image)
    features ={}
    gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
    
    # Thresholding to get binary silhouette
    ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
    # Iterate through contours
    for contour in contours:
        # Calculate area and perimeter
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        
        # Calculate bounding box
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h if h != 0 else 0.0
        
        # Calculate solidity (area / convex hull area)
        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        solidity = float(area) / hull_area if hull_area != 0 else 0.0

        features['area'] = area
        features['perimeter'] = perimeter
        features['aspect_ratio'] = aspect_ratio
        features['solidity'] = solidity

    return features
            

In [25]:
# load prebuilt Resnet model 

def make_deeplab(device):
    deeplab = deeplabv3_resnet101(pretrained=True).to(device)
    deeplab.eval()
    return deeplab

device = torch.device("cpu")
deeplab = make_deeplab(device)

/Users/acehunter/PycharmProjects/BodyScan/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/acehunter/PycharmProjects/BodyScan/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
# resize images

def preprocess(f_name):
    img_orig = cv2.imread(f_name, 1)
    k = min(1.0, 1024/max(img_orig.shape[0], img_orig.shape[1]))
    img = cv2.resize(img_orig, None, fx=k, fy=k, interpolation=cv2.INTER_LANCZOS4)
    return img

In [27]:
# Generate masked images

deeplab_preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def apply_deeplab(deeplab, img, device):
    input_tensor = deeplab_preprocess(img)
    input_batch = input_tensor.unsqueeze(0)
    with torch.no_grad():
        output = deeplab(input_batch.to(device))['out'][0]
    output_predictions = output.argmax(0).cpu().numpy()
    return (output_predictions == 15)

mask_front = apply_deeplab(deeplab, preprocess("front.jpeg"), device)


In [28]:
# Buff to store image temp

def create_buff(mask):
    buf = io.BytesIO()
    plt.imsave(buf, mask, cmap="gray", format="png")
    buf.seek(0)
    return buf

In [29]:
# Access the image stored in buff

def access_stored_image(buffer):
    # Open the image from the BytesIO buffer
    image = Image.open(buffer)
    #image.show()  # Display the image (optional)
    return image

# Call the method with the stored buffer
stored_image = access_stored_image(create_buff(mask_front))

In [30]:
extract_silhouette_features_for_inference(stored_image)

{'area': 147579.0,
 'perimeter': 4205.478474617004,
 'aspect_ratio': 0.4346895074946467,
 'solidity': 0.5697766898830943}

In [31]:
mask_left = apply_deeplab(deeplab, preprocess("left.jpeg"), device)

In [32]:
stored_image_left = access_stored_image(create_buff(mask_left))

In [33]:
extract_silhouette_features_for_inference(stored_image_left)

{'area': 95585.0,
 'perimeter': 2437.1311532258987,
 'aspect_ratio': 0.1699481865284974,
 'solidity': 0.6904734386061228}

In [34]:
dict1 = extract_silhouette_features_for_inference(stored_image)
dict2 = extract_silhouette_features_for_inference(stored_image_left)

In [35]:
combined_dict = {
    'area': dict1['area'],
    'perimeter': dict1['perimeter'],
    'aspect_ratio': dict1['aspect_ratio'],
    'solidity': dict1['solidity'],
    'area_left': dict2['area'],
    'perimeter_left': dict2['perimeter'],
    'aspect_ratio_left': dict2['aspect_ratio'],
    'solidity_left': dict2['solidity']
}

# Convert the combined dictionary to a DataFrame
df_inference = pd.DataFrame([combined_dict])
df_inference.head(5)

area    perimeter  aspect_ratio  solidity  area_left  perimeter_left  \
0  147579.0  4205.478475       0.43469  0.569777    95585.0     2437.131153   

   aspect_ratio_left  solidity_left  
0           0.169948       0.690473

In [36]:
# height and weight of subject

height = 182.88
weight = 84.44

In [37]:
# merge with image metrics

df_inference['weight_kg'] = 84.44
df_inference['height'] = 182.88
df_inference['gender'] = 1

In [38]:
df_inference.head(5)

area    perimeter  aspect_ratio  solidity  area_left  perimeter_left  \
0  147579.0  4205.478475       0.43469  0.569777    95585.0     2437.131153   

   aspect_ratio_left  solidity_left  weight_kg  height  gender  
0           0.169948       0.690473      84.44  182.88       1

In [39]:
# normalize wrt test data

mean = scaler.mean_
scale = scaler.scale_

In [40]:
df_inference['area'] = (df_inference['area'] - mean[0])/scale[0]
df_inference['height'] = (df_inference['height'] - mean[1])/scale[1]
df_inference['weight_kg'] = (df_inference['weight_kg'] - mean[2])/scale[2]
df_inference['perimeter'] = (df_inference['perimeter'] - mean[3])/scale[3]
df_inference['aspect_ratio'] = (df_inference['aspect_ratio'] - mean[4])/scale[4]
df_inference['solidity'] = (df_inference['solidity'] - mean[5])/scale[5]
df_inference['area_left'] = (df_inference['area_left'] - mean[6])/scale[6]
df_inference['perimeter_left'] = (df_inference['perimeter_left'] - mean[7])/scale[7]
df_inference['aspect_ratio_left'] = (df_inference['aspect_ratio_left'] - mean[8])/scale[8]
df_inference['solidity_left'] = (df_inference['solidity_left'] - mean[9])/scale[9]

In [41]:
df_inference.head(5)

area  perimeter  aspect_ratio  solidity  area_left  perimeter_left  \
0  1.929789   1.606774     -0.996416  1.166502   0.739772         0.41119   

   aspect_ratio_left  solidity_left  weight_kg   height  gender  
0          -0.316614       0.056371   0.569833  1.15047       1

In [42]:
# Predict Wrist 

wrist_circumfurance = model_wrist.predict(df_inference)

In [43]:
# Predict Waist

waist_circumfurance = model_waist.predict(df_inference)

In [44]:
# Predict hip

hip_circumfurance = model_hip.predict(df_inference)

In [45]:
# calc neck cir from wrist cir

neck_circumference= 2.64+1.94 * wrist_circumfurance

In [46]:
print(f'Neck:{neck_circumference}')
print(f'Waist:{waist_circumfurance}')
print(f'Hip:{hip_circumfurance}')

Neck:[[37.71204564]]
Waist:[[94.56270592]]
Hip:[[105.99954325]]


In [47]:
def calculate_final_metrics(sex, neck_circumference, waist_circumference, hip_circumference, height, weight):

    def calculate_bfp(sex, neck_circumference, waist_circumference, hip_circumference, height):
        """
        Calculate Body Fat Percentage using the Navy Body Fat formula.
        """
        if sex.lower() == 'male':
            bfp = 86.010 * math.log10(waist_circumference - neck_circumference) - 70.041 * math.log10(height) + 36.76
        elif sex.lower() == 'female':
            bfp = 163.205 * math.log10(waist_circumference + hip_circumference - neck_circumference) - 97.684 * math.log10(height) - 78.387
        else:
            raise ValueError("Sex must be 'male' or 'female'")
        return bfp

    def classify_fat_types(weight, bfp):
        """
        Classify fat types into essential, beneficial, and unbeneficial fats.
        """
        fm = weight * (bfp / 100)
        essential_fat = 0.05 * weight  # 5% of total body weight for essential fat
        beneficial_fat = 0.15 * weight  # 15% of total body weight for beneficial fat
        unbeneficial_fat = fm - (essential_fat + beneficial_fat)
        return essential_fat, beneficial_fat, unbeneficial_fat

    def calculate_lean_mass(weight, fm):
        """
        Calculate Lean Mass.
        """
        return weight - fm

    def calculate_indices(lean_mass, fm, height):
        """
        Calculate Lean Mass Index (LMI) and Fat Mass Index (FMI).
        """
        height_m = height / 100  # convert height to meters
        lmi = lean_mass / (height_m ** 2)
        fmi = fm / (height_m ** 2)
        return lmi, fmi

    def calculate_rmr(lean_mass):
        """
        Calculate Resting Metabolic Rate (RMR) using the Katch-McArdle formula.
        """
        return 370 + (21.6 * lean_mass)

    # Calculations
    bfp = calculate_bfp(sex, neck_circumference, waist_circumference, hip_circumference, height)
    fm = weight * (bfp / 100)
    essential_fat, beneficial_fat, unbeneficial_fat = classify_fat_types(weight, bfp)
    lean_mass = calculate_lean_mass(weight, fm)
    lmi, fmi = calculate_indices(lean_mass, fm, height)
    rmr = calculate_rmr(lean_mass)

    # Print Results
    print(f"Body Fat Percentage (BFP): {bfp:.2f}%")
    print(f"Essential Fat: {essential_fat:.2f} kg")
    print(f"Beneficial Fat: {beneficial_fat:.2f} kg")
    print(f"Unbeneficial Fat: {unbeneficial_fat:.2f} kg")
    print(f"Lean Mass: {lean_mass:.2f} kg")
    print(f"Lean Mass Index (LMI): {lmi:.2f} kg/m^2")
    print(f"Fat Mass Index (FMI): {fmi:.2f} kg/m^2")
    print(f"Resting Metabolic Rate (RMR): {rmr:.2f} kcal/day")


In [48]:
calculate_final_metrics('male', neck_circumference, waist_circumfurance, hip_circumfurance, height, weight)

Body Fat Percentage (BFP): 29.24%
Essential Fat: 4.22 kg
Beneficial Fat: 12.67 kg
Unbeneficial Fat: 7.80 kg
Lean Mass: 59.75 kg
Lean Mass Index (LMI): 17.86 kg/m^2
Fat Mass Index (FMI): 7.38 kg/m^2
Resting Metabolic Rate (RMR): 1660.59 kcal/day


/var/folders/02/vgb90thx747_z8w7ph05k5tw0000gn/T/ipykernel_91511/2588190403.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  bfp = 86.010 * math.log10(waist_circumference - neck_circumference) - 70.041 * math.log10(height) + 36.76
